# Création d'un dictionnaire à partir d'un fichier csv

In [74]:
import csv
from datetime import datetime
import os
import glob

def create_timestamp(date):
    dt = datetime(int(date[0:4]), int(date[5:7]), int(date[8:10]), 
                   int(date[11:13]), int(date[14:16]), int(date[17:19]))
    
    dt = datetime.timestamp(dt)
    return dt


def filename_from_path(file):
    
    begin = len(file)-1
    
    while file[begin] != ".":
        begin -= 1
        
    end = begin
    
    while file[begin] != "\\":
          begin -= 1
          
    begin += 1
    return file[begin:end]
    
    

notebook_path = os.path.abspath("Notebook.ipynb")
chemin = os.path.join(os.path.dirname(notebook_path), "results/assets")

paths = glob.glob(chemin+'/*')

data = []

for i in range(0,4):
    
    name = filename_from_path(paths[i])
    
    with open(paths[i], newline='') as csvfile:
        reader = csv.DictReader(csvfile)

        for row in reader:
            point = {}      
            point['asset'] = name
            point['recorded_at'] = create_timestamp(row['recorded_at'])
            
            coord = []
            coord.append(float(row['longitude']))
            coord.append(float(row['latitude']))
            point['coordinates'] = coord
            
            data.append(point)
        
    data = sorted(data, key=lambda point: point['recorded_at'])   # tri selon recorded_at

    
print('done')

done


# Importation du dictionnaire dans la base de données 'munic'

In [3]:
!pip install pymongo

In [75]:
import pymongo

client = pymongo.MongoClient('localhost',27017)
mydb = client["munic"]
collection = mydb["assets"]

print('insert')
collection.insert_many(data)
print('done')


insert
done


# Index géo spatial

In [76]:
from pymongo import GEOSPHERE
# db.assets.createIndex( { location : "2dsphere" } )

client = pymongo.MongoClient('localhost')
collection = client.munic.assets

resp = collection.create_index([("coordinates",GEOSPHERE)])
print ("index response:", resp)

index response: coordinates_2dsphere


# Lecture dans la base

In [7]:
!pip install pandas

In [83]:
import pandas as pd

client = pymongo.MongoClient('localhost')
collection = client.munic.assets

cursor = collection.find()
entries = list(cursor)

entries[:]

#df = pd.DataFrame(entries)
#df.head()

[{'_id': ObjectId('5f154a9d2b859f7d9012d02c'),
  'asset': '14256004535292',
  'recorded_at': 1583145392.0,
  'coordinates': [45.5857, -73.7699]},
 {'_id': ObjectId('5f154a9d2b859f7d9012d02d'),
  'asset': '14256004535292',
  'recorded_at': 1583145412.0,
  'coordinates': [45.5858, -73.7698]},
 {'_id': ObjectId('5f154a9d2b859f7d9012d02e'),
  'asset': '14256004535292',
  'recorded_at': 1583145413.0,
  'coordinates': [45.5858, -73.7698]},
 {'_id': ObjectId('5f154a9d2b859f7d9012d02f'),
  'asset': '14256004535292',
  'recorded_at': 1583145413.0,
  'coordinates': [45.5858, -73.7698]},
 {'_id': ObjectId('5f154a9d2b859f7d9012d030'),
  'asset': '14256004535292',
  'recorded_at': 1583145453.0,
  'coordinates': [45.5858, -73.7698]},
 {'_id': ObjectId('5f154a9d2b859f7d9012d031'),
  'asset': '14256004535292',
  'recorded_at': 1583145476.0,
  'coordinates': [45.5858, -73.7698]},
 {'_id': ObjectId('5f154a9d2b859f7d9012d032'),
  'asset': '14256004535292',
  'recorded_at': 1583145776.0,
  'coordinates': 

ERROR: Could not find a version that satisfies the requirement 2dsphere (from versions: none)
ERROR: No matching distribution found for 2dsphere


In [84]:
import pymongo
import pprint
client = pymongo.MongoClient('localhost')
collection = client.munic.assets
a=1.86411/3963.2
i=0
x=45.5694
y=-73.9107
for doc in collection.find( {  "coordinates":
                         { "$geoWithin" :
                           { "$centerSphere" :
                              [ [x,y],a] }
                      } })h:
    i=i+1
print(i)


54


In [49]:
!pip install notebook